In [1]:
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from pgmpy.estimators import MmhcEstimator
from pgmpy.estimators import BDeuScore
from pgmpy.estimators import HillClimbSearch

In [92]:
os.chdir("C:/Users/Mitsu/Desktop/AISTATS/230930_MMHC/")
def input_data(filename, num_of_var):
    tmp = pd.read_csv("sample_data_"+str(num_of_var)+"_"+filename, sep=",", encoding="utf-8")
    del tmp["Unnamed: 0"]
    return tmp

In [98]:
def mat_(tmp):
    if "e" in tmp:
        var = tmp.split("e")[1].split("(")[0]
        matrix = "E"
    elif "x" in tmp:
        if "t-1" in tmp:
            var = tmp.split("x")[1].split("(")[0]
            matrix = "B1"
        elif "t" in tmp:
            var = tmp.split("x")[1].split("(")[0]
            matrix = "B0"
    else:
            pass
    return var, matrix

In [99]:
def Measurements(coeff_pred, coeff_truth):
    TP,TN,FP,FN=0,0,0,0
    for i in range(coeff_pred.shape[0]):
        for j in range(coeff_pred.shape[1]):
            if coeff_pred[i][j] == coeff_truth[i][j]:
                if coeff_pred[i][j]==1.0:
                    # print("True positive", coeff_pred[i][j])
                    TP += 1
                elif coeff_pred[i][j]==0.0:
                    # print("True negative", coeff_pred[i][j])
                    TN += 1
            else:
                if coeff_pred[i][j]==1.0:
                    # print("False positive", coeff_pred[i][j])
                    FP += 1
                elif coeff_pred[i][j]==0.0:
                    # print("False negative", coeff_pred[i][j])
                    FN += 1

    Accuracy = (TP + TN) / (TP + FP + FN + TN)
    # Precision = TP / (TP + FP)
    Recall = TP / (TP+FN)
    F1 = TP / (TP + 1/2 * (FP + FN))
    MyScore = 1 / ( 1 + (FP + FN) )
    MyScore2 = (FP + FN)  / (TP + FP + FN + TN)
    return Accuracy, Recall, F1, MyScore, MyScore2 # Precision, 

In [100]:
num_of_var_list = [2,3,4]
for num_of_var in num_of_var_list:
    print("num_of_var: ",num_of_var)
    train_gen = input_data(filename="train_gen.csv", num_of_var=num_of_var)
    train_dis = input_data(filename="train_dis.csv", num_of_var=num_of_var)
    test_gen = input_data(filename="test_gen.csv", num_of_var=num_of_var)
    test_dis = input_data(filename="test_dis.csv", num_of_var=num_of_var)
        
    data = train_gen
    # data = data[data.columns[0:num_of_var*2]]
    
    time_sta = time.perf_counter()
    
    mmhc = MmhcEstimator(data)
    skeleton = mmhc.mmpc()
    print("Part 1) Skeleton: ", skeleton.edges())
    
    hc = HillClimbSearch(data)
    model = hc.estimate(tabu_length=10, white_list=skeleton.to_directed().edges(), scoring_method=BDeuScore(data))
    print("Part 2) Model:    ", model.edges())
    
    time_end = time.perf_counter()
    tim = time_end- time_sta
    print(tim,"sec.")
    ###################################################
    
    B0_pred = np.zeros([num_of_var, num_of_var])
    B1_pred = np.zeros([num_of_var, num_of_var])
    E_pred = np.zeros([num_of_var, num_of_var])
    for edge in np.array(model.edges()):
        edge_from = edge[0]
        mat_from = mat_(edge_from)
        edge_to = edge[1]
        mat_to = mat_(edge_to)
        if mat_from[1] == "B0":
            if mat_to[1] == "B0":
                B0_pred[int(mat_from[0]), int(mat_to[0])] = 1.0
                
        elif mat_from[1] == "B1":
            if mat_to[1] == "B0":
                B1_pred[int(mat_from[0]), int(mat_to[0])] = 1.0
            
        elif mat_from[1] == "E":
            if mat_to[1] == "B0":
                E_pred[int(mat_from[0]), int(mat_to[0])] = 1.0
    
    #############################################################
    
    tmp = pd.read_csv("sample_data_"+str(num_of_var)+".csv", sep=",", encoding="utf-8")
    del tmp["Unnamed: 0"]
    B0 = np.array(tmp[0:num_of_var])
    B1 = np.array(tmp[num_of_var : num_of_var * 2])
    E = np.diag([1.0]*num_of_var)
    coeff_truth = np.concatenate([B0, B1, E])
    coeff_truth = np.where(abs(coeff_truth)>0.001, 1, 0)
    coeff_pred = np.concatenate([B0_pred, B1_pred, E_pred])
    Accuracy, Recall, F1, MyScore, MyScore2 = Measurements(coeff_pred, coeff_truth)
    df = pd.DataFrame({
        "Num_of_var": [num_of_var],
        "Accuracy": [Accuracy],
        # "Precision": [Precision],
        "Recall": [Recall],
        "F1": [F1],
        "Error rate": [MyScore2], # 1 - Accuracy （全要素中、何割間違えたか）
        "1/(1+FP+FN)": [MyScore],
        "Time": [tim],
        # "DTW": DTW_dict_list,
        # "DTW_mean": DTW_mean_list,
        # "Coeff_result": [np.concatenate([B0_pred,B1_pred])],
        # "Coeff_result_std": Coeff_result_std_list,
        "Coeff_Truth": [coeff_truth],
        "Coeff_result_probability": [coeff_pred]
    })
    df.to_csv("Results_MMHC_"+str(num_of_var)+".csv", sep=",", encoding="utf-8")
    print(df)


num_of_var:  2
Part 1) Skeleton:  [('x0(t)', 'e1(t)'), ('x1(t)', 'e0(t)'), ('x1(t)', 'x1(t-1)'), ('x0(t-1)', 'e0(t)'), ('e0(t)', 'e1(t)')]


  0%|          | 0/1000000 [00:00<?, ?it/s]

Part 2) Model:     [('x1(t)', 'e0(t)'), ('x1(t-1)', 'x1(t)'), ('e0(t)', 'x0(t-1)'), ('e0(t)', 'e1(t)'), ('e1(t)', 'x0(t)')]
107.60927210003138 sec.
   Num_of_var  Accuracy  Recall   F1  Error rate  1/(1+FP+FN)        Time  \
0           2  0.583333     0.0  0.0    0.416667     0.166667  107.609272   

                                        Coeff_Truth  \
0  [[0, 0], [0, 0], [0, 1], [0, 0], [1, 0], [0, 1]]   

                            Coeff_result_probability  
0  [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 1.0...  
num_of_var:  3
Part 1) Skeleton:  [('x0(t)', 'x0(t-1)'), ('x0(t)', 'e1(t)'), ('x1(t)', 'e2(t)'), ('x1(t)', 'e1(t)'), ('x2(t)', 'e2(t)'), ('x2(t)', 'x0(t-1)'), ('x2(t)', 'e1(t)'), ('x0(t-1)', 'x2(t-1)'), ('x0(t-1)', 'e2(t)'), ('x0(t-1)', 'x1(t-1)'), ('x1(t-1)', 'e0(t)'), ('x1(t-1)', 'e1(t)'), ('x2(t-1)', 'e2(t)'), ('e0(t)', 'e1(t)')]


  0%|          | 0/1000000 [00:00<?, ?it/s]

Part 2) Model:     [('x1(t)', 'e1(t)'), ('x1(t)', 'e2(t)'), ('x2(t-1)', 'x0(t-1)'), ('e0(t)', 'x1(t-1)'), ('e1(t)', 'x0(t)'), ('e1(t)', 'x2(t)'), ('e1(t)', 'e0(t)'), ('e2(t)', 'x2(t-1)')]
1641.1262395000085 sec.
   Num_of_var  Accuracy  Recall   F1  Error rate  1/(1+FP+FN)        Time  \
0           3  0.666667     0.0  0.0    0.333333          0.1  1641.12624   

                                         Coeff_Truth  \
0  [[0, 1, 0], [0, 0, 1], [0, 0, 0], [1, 0, 0], [...   

                            Coeff_result_probability  
0  [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...  
num_of_var:  4
Part 1) Skeleton:  [('x0(t)', 'x1(t-1)'), ('x0(t)', 'x0(t-1)'), ('x0(t)', 'e1(t)'), ('x0(t)', 'x3(t)'), ('x1(t)', 'e3(t)'), ('x1(t)', 'e0(t)'), ('x2(t)', 'x0(t-1)'), ('x2(t)', 'e0(t)'), ('x3(t)', 'x0(t-1)'), ('x3(t)', 'e0(t)'), ('x3(t)', 'x3(t-1)'), ('x3(t)', 'e1(t)'), ('x0(t-1)', 'x2(t-1)'), ('x0(t-1)', 'e0(t)'), ('x0(t-1)', 'e1(t)'), ('x0(t-1)', 'e3(t)'), ('x1(t-1)', 'x3(t-1)'), ('x1(t-1)',

  0%|          | 0/1000000 [00:00<?, ?it/s]

Part 2) Model:     [('x3(t)', 'e1(t)'), ('x3(t)', 'x0(t-1)'), ('x1(t-1)', 'x0(t)'), ('x1(t-1)', 'x3(t-1)'), ('x2(t-1)', 'e0(t)'), ('x2(t-1)', 'x1(t-1)'), ('e0(t)', 'e3(t)'), ('e0(t)', 'x3(t)'), ('e0(t)', 'x2(t)'), ('e3(t)', 'x1(t)'), ('e3(t)', 'e2(t)')]
17334.65415810002 sec.
   Num_of_var  Accuracy  Recall   F1  Error rate  1/(1+FP+FN)          Time  \
0           4  0.666667     0.0  0.0    0.333333     0.058824  17334.654158   

                                         Coeff_Truth  \
0  [[0, 0, 1, 0], [0, 0, 1, 1], [0, 0, 0, 1], [0,...   

                            Coeff_result_probability  
0  [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...  
num_of_var:  5


KeyboardInterrupt: 

In [15]:
# pd.DataFrame(np.random.randint(0, 3, size=(2500, 8)), columns=list('ABCDEFGH'))

,A,B,C,D,E,F,G,H
0,0,2,0,0,1,2,0,2
1,0,2,0,1,1,1,0,2
2,2,0,2,0,0,0,1,1
3,0,0,2,1,0,2,2,1
4,1,1,0,1,2,1,1,1
...,...,...,...,...,...,...,...,...
2495,0,0,1,2,2,0,0,0
2496,0,2,1,1,1,0,0,0
2497,0,2,1,1,2,1,0,0
2498,0,1,1,2,1,0,2,1
